**Задание 1.**

Мы будем работать с вложенным словарем кинокритиков и выставленных ими оценок для небольшого набора данных о фильмах.

Какой здесь могла бы быть неперсонализированная рекомендация? Попробуйте написать свою реализацию неперсонализированной рекомендации.

In [ ]:
import warnings
warnings.filterwarnings("ignore")

import pprint
pp = pprint.PrettyPrinter(indent=4)

In [ ]:
# Словарь кинокритиков и выставленных ими оценок
critics={'Lisa Rose': {'Lady in the Water': 2.5, 'Snakes on a Plane': 3.5,
 'Just My Luck': 3.0, 'Superman Returns': 3.5, 'You, Me and Dupree': 2.5, 
 'The Night Listener': 3.0},
'Gene Seymour': {'Lady in the Water': 3.0, 'Snakes on a Plane': 3.5, 
 'Just My Luck': 1.5, 'Superman Returns': 5.0, 'The Night Listener': 3.0, 
 'You, Me and Dupree': 3.5}, 
'Michael Phillips': {'Lady in the Water': 2.5, 'Snakes on a Plane': 3.0,
 'Superman Returns': 3.5, 'The Night Listener': 4.0},
'Claudia Puig': {'Snakes on a Plane': 3.5, 'Just My Luck': 3.0,
 'The Night Listener': 4.5, 'Superman Returns': 4.0, 
 'You, Me and Dupree': 2.5},
'Mick LaSalle': {'Lady in the Water': 3.0, 'Snakes on a Plane': 4.0, 
 'Just My Luck': 2.0, 'Superman Returns': 3.0, 'The Night Listener': 3.0,
 'You, Me and Dupree': 2.0}, 
'Jack Matthews': {'Lady in the Water': 3.0, 'Snakes on a Plane': 4.0,
 'The Night Listener': 3.0, 'Superman Returns': 5.0, 'You, Me and Dupree': 3.5},
'Toby': {'Snakes on a Plane':4.5,'You, Me and Dupree':1.0,'Superman Returns':4.0}}

Первой рекомендацией может быть самый популярный фильм, т.е. фильм, который имеет больше всего оценок.

In [ ]:
films = []

for i in critics.keys():
  for j in critics[i]:
    films.append(j)

count = {}
for i in films:
  if i not in count.keys():
    count[i] = films.count(i)

pp.pprint(sorted(count.items(), key = lambda x: x[1], reverse = True))

[   ('Snakes on a Plane', 7),
    ('Superman Returns', 7),
    ('You, Me and Dupree', 6),
    ('The Night Listener', 6),
    ('Lady in the Water', 5),
    ('Just My Luck', 4)]


Два фильма одинаково популярны: "Snakes on a Plane" и "Superman Returns".

Воспользуемся другим способом и порекомендуем фильм с самой высокой средней оценкой. 


In [ ]:
avrdict = {}

for j in count.keys():
    avr = 0
    for i in critics.keys():  
      if j in critics[i]:
        avr += critics[i][j]
        avrdict.update({j: avr})
        
for i in avrdict:
  avrdict.update({i : avrdict[i]/count[i]})
pp.pprint(sorted(avrdict.items(), key = lambda x: x[1], reverse = True))

[   ('Superman Returns', 4.0),
    ('Snakes on a Plane', 3.7142857142857144),
    ('The Night Listener', 3.4166666666666665),
    ('Lady in the Water', 2.8),
    ('You, Me and Dupree', 2.5),
    ('Just My Luck', 2.375)]


Здесь самую высокую оценку набирает "Superman Returns", его в конечном счете и порекомендуем.

**Задание 2.**

Коллаборативная фильтрация по схожести образцов.

Чтобы сравнивать образцы, нужно первым делом построить полный набор данных о похожих образцах. В первую очередь, нам нужно переделать словарь предпочтений критиков таким образом, чтобы теперь был словарь фильмов и их оценок.



In [ ]:
# обращаем матрицу предпочтений, чтобы строки соответствовали образцам
def transformPrefs(critics_dict):

    # инициилизируем новый словарь с фильмами и их оценками
    result = {}

    # для каждого критика в словаре
    for person in critics_dict:

        # для каждого оцененного им фильма
        for item in critics_dict[person]:

            # добавляем такой объект в словарь
            result.setdefault(item, {})
            # меняеи местами человека и предмет
            result[item][person] = critics_dict[person][item]
    return result

Далее для каждого фильма посчитать наиболее похожие образцы и коэффициенты похожести для них. Для подсчета схожести будем использовать евклидово расстояние.

Из задачи коллаборативной фильтрации по схожести пользователей нам понадобятся функции similar_films, sim_distance, top_Matches.

In [ ]:
def similar_films(critics_dict, person1, person2):
    # создаем пустой список фильмов с названием 
    sim_film = []
    for i in critics_dict[person1]: 
      if i in critics_dict[person2]:
        sim_film.append(i)
    # для всех фильмов у person1, если такой фильм оценивался person 2, добавляем этот фильм в список sim_film

    return sim_film

In [ ]:
def sim_distance(critics_dict, person1, person2):
    # используем написанную функцию для получения фильмов, оцененных обоими критиками
    sim_films = similar_films(critics_dict, person1, person2)

    # если нет ни одной общей оценки, то есть длина sim_film равна 0 - вернуть 0
    if len(sim_films) == 0:
        return 0

    # для каждого фильма из sim_film посчитаем евклидово расстояние и просуммируем все полученные значения sum_of_euclead_dist
    sum_of_euclead_dist = 0
    for film in sim_films:
      sum_of_euclead_dist = sum_of_euclead_dist + ( critics_dict[person1][film] - critics_dict[person2][film] ) **2 

    # считаем похожесть 
    score = 1 / (1 + sum_of_euclead_dist)

    return score

In [ ]:
def topMatches(critics_dict, person, n=5, similarity=sim_distance):

    top = []
    for user in critics_dict:
      if user != person:
          sim = similarity(critics_dict, user, person)
          top.append((sim, user))
    top.sort(reverse = True)
    return top[:n]

Реализуйте функцию calculateSimilarItems(), которая делает сразу оба шага. На 
вход её нужно подать словарь с оценками критиков critics и число образцов, которые мы считаем наиболее похожими (остальные не выводим). Например, так calculateSimilarItems(critics, n=10).


In [ ]:
# Создать словарь, содержащий для каждого образца те образцы, которые больше всего похожи на него.
def calculateSimilarItems(prefs, n = 10):

    films = transformPrefs(prefs)
    ranged_films = {}

    for film in films:
      similarity_scores = topMatches(films, film, n, similarity=sim_distance)
      ranged_films[film] = similarity_scores

    return ranged_films  

In [ ]:
itemsim = calculateSimilarItems(critics, n=3)

In [ ]:
pp.pprint(itemsim)

{   'Just My Luck': [   (0.2222222222222222, 'Lady in the Water'),
                        (0.18181818181818182, 'You, Me and Dupree'),
                        (0.15384615384615385, 'The Night Listener')],
    'Lady in the Water': [   (0.4, 'You, Me and Dupree'),
                             (0.2857142857142857, 'The Night Listener'),
                             (0.2222222222222222, 'Snakes on a Plane')],
    'Snakes on a Plane': [   (0.2222222222222222, 'Lady in the Water'),
                             (0.18181818181818182, 'The Night Listener'),
                             (0.16666666666666666, 'Superman Returns')],
    'Superman Returns': [   (0.16666666666666666, 'Snakes on a Plane'),
                            (0.10256410256410256, 'The Night Listener'),
                            (0.09090909090909091, 'Lady in the Water')],
    'The Night Listener': [   (0.2857142857142857, 'Lady in the Water'),
                              (0.18181818181818182, 'Snakes on a Plane'),
      

Получили вложенный словарь с фильмами и их похожестью на другие фильмы.

Теперь мы готовы выдавать рекомендации, пользуясь словарем данных о схожести образцов. 

По аналогии с тем, что мы уже делали, необходимо получить список всех образцов, которым пользователь выставлял оценки, найти похожие и взвесить их с учетом коэффициентов подобия (умножить оценку на коэффициент подобия).

Реализуйте этот механизм в функции getRecommendedItems(). 

In [ ]:
def getRecommendedItems(prefs, itemMatch, user):

    # инициализируем словари
    userRatings = prefs[user]
    scores = {}
    totalSim = {}

    # цикл по образцам, оцененным данным пользователем (хранятся в userRatings)
    for (item, rating) in userRatings.items():

        # цикл по образцам, похожим на данный (хранятся в словаре itemMatch)
        for (similarity, item2) in itemMatch[item]:
            # пропускаем объект, если пользователь уже оценивал данный образец
            if item2 in userRatings:
                continue
            # взвешенная суммы оценок, умноженных на коэффициент подобия
            scores.setdefault(item2, 0)
            scores[item2] = scores[item2] + similarity * rating
            # сумма всех коэффициентов подобия
            totalSim.setdefault(item2, 0)
            totalSim[item2] = totalSim[item2] + similarity
            if totalSim[item2] == 0:
                totalSim[item2] = 0.0000001  # чтобы избежать деления на ноль
    # делим каждую итоговую оценку на взвешенную сумму, чтобы вычислить среднее
    rankings = [(score / totalSim[item], item) for item, score in scores.items()]

    # возвращает список rankings, отсортированный по убыванию
    rankings.sort(reverse=True)
    return rankings

In [ ]:
getRecommendedItems(critics, itemsim, 'Toby')

[(3.182634730538922, 'The Night Listener'),
 (2.4730878186968837, 'Lady in the Water'),
 (1.0, 'Just My Luck')]

Рекомендация получилась так себе. Ведь это просто ранжированный список фильмов, которые он не смотрел. Давайте в рекомендацию будем вклчать только те фильмы, за которые прогнозируемый средний балл превышает средний балл за те фильмы, которые уже оценил критик.

In [ ]:
def critic_scores(critics_dict, name):
    
    avr = 0
    for i in critics_dict[name]:
        avr += critics_dict[name][i]
    return avr/len(critics_dict[name])  

critic_scores(critics, "Toby")

3.1666666666666665

In [ ]:
def averageRecommended(critics_dict, name):
  recommended_films = []
  
  for i in getRecommendedItems(critics_dict, itemsim, name):
    if i[0] > critic_scores(critics_dict, name):
      recommended_films.append(i[1])

  if len(recommended_films) !=0:
     return recommended_films
  else:
     print(f'Nothing new for you, {name}. Check in later!')

In [ ]:
averageRecommended(critics, "Toby")

['The Night Listener']

In [ ]:
averageRecommended(critics, "Lisa Rose")

Nothing new for you, Lisa Rose. Check in later!
